In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_parquet("silver_fhv_info.parquet")

In [3]:
df

,PickUpID,DropOffID,dispatching_base_num,Affiliated_base_number
0,None,F202301466393,B01437,B03404
1,None,F202301893606,B00856,B03152
2,F202301280562,F202301241371,B03284,
3,None,F202301304021,B00856,B02932
4,None,F202301908866,B00821,B00821
...,...,...,...,...
1114315,None,None,B00013,B00014
1114316,None,None,B00411,B00053
1114317,None,None,B00412,B00412
1114318,None,F202302728594,B01406,B02747


In [4]:
df = pd.read_parquet("D:/OneDrive - VNU-HCMUS/Introduction to AI/Tuần 3/silver_fhv_dropoff.parquet")
df.head(20)

,dropOff_datetime,DOlocationID,DropOffID
0,2023-01-01 00:31:59,39.0,202301618211
1,2023-01-01 01:39:04,119.0,202301663020
2,2023-01-01 01:25:57,252.0,202301841072
3,2023-01-01 01:59:44,71.0,202301978766
4,2023-01-01 02:23:07,89.0,202301375755
5,2023-01-01 01:31:25,239.0,202301886204
6,2023-01-01 02:07:01,147.0,202301702041
7,2023-01-01 01:41:38,148.0,202301136878
8,2023-01-01 01:43:29,179.0,202301909146
9,2023-01-01 01:47:02,74.0,202301339706
